In [1]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [2]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
417,THE FEELING of the need to have someone play t...,positive
893,"Much like the comedy duo of its title, ""The Su...",positive
54,As someone who likes chase scenes and was real...,negative
310,"Why, o' WHY! ...did I pick this one up? Well.....",negative
677,Set in and near a poor working class town in t...,positive


In [3]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [5]:
df = normalize_text(df)
df.head()

,review,sentiment
417,feeling need someone play role arbiter public ...,positive
893,much like comedy duo title sunshine boy become...,positive
54,someone like chase scene really intrigued fasc...,negative
310,pick one well needed brainer summer heat cover...,negative
677,set near poor working class town mountain rura...,positive


In [6]:
df['sentiment'].value_counts()

sentiment
negative    253
positive    247
Name: count, dtype: int64

In [7]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [8]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
417,feeling need someone play role arbiter public ...,1
893,much like comedy duo title sunshine boy become...,1
54,someone like chase scene really intrigued fasc...,0
310,pick one well needed brainer summer heat cover...,0
677,set near poor working class town mountain rura...,1


In [9]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [14]:
vectorizer = CountVectorizer(max_features=50)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [16]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/i2pandit/MLops_Text_Classification.mlflow')
dagshub.init(repo_owner='i2pandit', repo_name='MLops_Text_Classification', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


2025-03-14 15:43:31,308 - INFO - HTTP Request: GET https://dagshub.com/api/v1/repos/i2pandit/MLops_Text_Classification "HTTP/1.1 200 OK"


Initialized MLflow to track repo "i2pandit/MLops_Text_Classification"

2025-03-14 15:43:31,318 - INFO - Initialized MLflow to track repo "i2pandit/MLops_Text_Classification"


Repository i2pandit/MLops_Text_Classification initialized!

2025-03-14 15:43:31,321 - INFO - Repository i2pandit/MLops_Text_Classification initialized!


<Experiment: artifact_location='mlflow-artifacts:/985db6ef7f0548d79e0ff97212f8b666', creation_time=1741980942569, experiment_id='0', last_update_time=1741980942569, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [17]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 50)
        mlflow.log_param("test_size", 0.20)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-03-14 15:43:54,710 - INFO - Starting MLflow run...
2025-03-14 15:43:54,840 - INFO - Logging preprocessing parameters...
2025-03-14 15:43:55,191 - INFO - Initializing Logistic Regression model...
2025-03-14 15:43:55,192 - INFO - Fitting the model...
2025-03-14 15:43:55,206 - INFO - Model training complete.
2025-03-14 15:43:55,207 - INFO - Logging model parameters...
2025-03-14 15:43:55,304 - INFO - Making predictions...
2025-03-14 15:43:55,305 - INFO - Calculating evaluation metrics...
2025-03-14 15:43:55,313 - INFO - Logging evaluation metrics...
2025-03-14 15:43:55,738 - INFO - Saving and logging the model...
2025/03/14 15:44:01 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025-03-14 15:44:04,461 - INFO - Model training and logging completed in 9.62 seconds.
2025-03-14 15:44:04,462 - INFO - Accuracy: 0.69
2025-03-14 15:44:04,462 - INFO - Precision: 0.

🏃 View run industrious-mare-286 at: https://dagshub.com/i2pandit/MLops_Text_Classification.mlflow/#/experiments/0/runs/3775c4b479614005bb44af0a6de7b180
🧪 View experiment at: https://dagshub.com/i2pandit/MLops_Text_Classification.mlflow/#/experiments/0
